# Pandas

Importación y carga del CSV (con cronómetro) y muestra las primeras 5 filas del dataset

In [1]:
import pandas as pd
import time

In [2]:
timer_start = time.time()   
df = pd.read_csv("dataset.csv", low_memory=False)
timer_end = time.time()
print("Pandas read_csv time:", timer_end - timer_start)
df.head()

Pandas read_csv time: 9.207297086715698


,Unnamed: 0,A,A.1,A.2,A.3,A.4,AAL,AAL.1,AAL.2,AAL.3,...,ZION,ZION.1,ZION.2,ZION.3,ZION.4,ZTS,ZTS.1,ZTS.2,ZTS.3,ZTS.4
0,NaN,open,high,low,close,volume,open,high,low,close,...,open,high,low,close,volume,open,high,low,close,volume
1,timestamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-09-11 09:30:00,NaN,NaN,NaN,NaN,NaN,44.01,44.05,44.01,44.01,...,42.05,42.05,42.04,42.04,26933.0,NaN,NaN,NaN,NaN,NaN
3,2017-09-11 09:31:00,65.5,65.5,65.41,65.46,29852.0,44.01,44.25,44.0,44.25,...,42.06,42.54,42.01,42.24,39292.0,65.33,65.39,64.96,65.11,38144.0
4,2017-09-11 09:32:00,65.4604,65.66,65.4604,65.66,3435.0,44.25,44.32,44.22,44.27,...,42.12,42.48,42.09,42.48,9683.0,65.115,65.23,65.075,65.12,4390.0


Resumen del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43148 entries, 0 to 43147
Columns: 2511 entries, Unnamed: 0 to ZTS.4
dtypes: object(2511)
memory usage: 826.6+ MB


Contar los valores faltantes en cada colummna, devolviendo una serie donde cada elemento es el recuento de nulos de la columna

In [5]:
df.isnull().sum()

Unnamed: 0       1
A             1606
A.1           1606
A.2           1606
A.3           1606
              ... 
ZTS            990
ZTS.1          990
ZTS.2          990
ZTS.3          990
ZTS.4          990
Length: 2511, dtype: int64

Renombramos la columna Timestamp y eliminamos la fila repetida. Remplazamos los valores que representan nulos por nan ya que pandas los reconoce mejor.
Despues convertimos la columna Timestamp a tipo fecha (datatime), tambien convertimos las columnas a tipo float limpiando comas de los miles.
Finalmente eliminamos filas sin el Timestamp valido y guardamos una copia para proximas operaciones


In [7]:
import numpy as np

#  renombrar la columna Timestamp y eliminar fila repetida si existe
df = df.rename(columns={df.columns[0]: "Timestamp"})
if str(df.iloc[0, 0]).strip().lower() == "timestamp":
    df = df.iloc[1:].reset_index(drop=True)

#remplazo los valores que representan nulos por nan ya que pandas los reconoce mejor
df = df.replace(
    to_replace=["NULL", "Null", "null", "NaN", "nan", ""],
    value=np.nan
)    
# convierto la columna Timestamp a tipo datetime
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# convierto las columnas numéricas a tipo float, limpiando espacios y comas de miles
value_cols = df.columns.drop("Timestamp")
df[value_cols] = (
    df[value_cols]
      .apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", ""), errors="coerce"))
)

# eliminar filas sin timestamp válido
df = df.dropna(subset=["Timestamp"])
df_limpio = df
df_pandasguar = df_limpio.copy()

reemplazamos con la mediana cada valor nan del dataset 

In [8]:

# 7) Imputar con **MEDIANA** columna a columna (más robusta que la media)
medianas = df[value_cols].median()
df[value_cols] = df[value_cols].fillna(medianas)
df.head()

,Timestamp,A,A.1,A.2,A.3,A.4,AAL,AAL.1,AAL.2,AAL.3,...,ZION,ZION.1,ZION.2,ZION.3,ZION.4,ZTS,ZTS.1,ZTS.2,ZTS.3,ZTS.4
2,2017-09-11 09:30:00,67.5000,67.519,67.4900,67.50,1801.0,44.01,44.05,44.0100,44.01,...,42.05,42.05,42.04,42.04,26933.0,71.290,71.31,71.270,71.29,2700.0
3,2017-09-11 09:31:00,65.5000,65.500,65.4100,65.46,29852.0,44.01,44.25,44.0000,44.25,...,42.06,42.54,42.01,42.24,39292.0,65.330,65.39,64.960,65.11,38144.0
4,2017-09-11 09:32:00,65.4604,65.660,65.4604,65.66,3435.0,44.25,44.32,44.2200,44.27,...,42.12,42.48,42.09,42.48,9683.0,65.115,65.23,65.075,65.12,4390.0
5,2017-09-11 09:33:00,65.6700,65.700,65.6200,65.69,700.0,44.28,44.46,44.2504,44.34,...,42.49,42.49,42.18,42.20,7844.0,71.290,71.31,71.270,71.29,2700.0
6,2017-09-11 09:34:00,65.6900,65.880,65.6800,65.88,2814.0,44.34,44.36,44.2200,44.24,...,42.28,42.38,42.23,42.38,6472.0,65.120,65.18,64.970,64.97,5515.0


Verificamos si quedan nulos 

In [9]:
df.isnull().sum()

Timestamp    0
A            0
A.1          0
A.2          0
A.3          0
            ..
ZTS          0
ZTS.1        0
ZTS.2        0
ZTS.3        0
ZTS.4        0
Length: 2511, dtype: int64

filtrado temporal y agrupamiento por fecha usando la librería Pandas. Además, mide el tiempo de ejecución del proceso para evaluar su rendimiento.
Se definen dos marcas de tiempo (t0 y t1) que servirán como límites para filtrar los datos dentro de ese intervalo horario.
Se inicia un contador de tiempo para medir cuánto tarda el proceso completo de filtrado y agrupamiento.
mask crea una máscara booleana que selecciona solo las filas cuyo valor en la columna "Timestamp" está dentro del rango entre t0 y t1.
Luego, df_f crea una copia de esas filas filtradas para trabajar de forma segura sin modificar el DataFrame original.
Se agrega una nueva columna llamada "date" que contiene únicamente la parte de la fecha (sin la hora) extraída de "Timestamp".
Esto facilita el agrupamiento posterior por día.
Calculamos cuantas filas quedaron despues del filtrado y agrupamos los datos por la columna date y tambien cuenta cuantas apariciones hay de cada fecha
finalmente ordena el resultado por el orden cronologico del indece y se detiene el temporizador 

In [10]:
import time
t0, t1 = pd.Timestamp("2017-09-11 09:30:00"), pd.Timestamp("2017-09-11 10:30:00")

tic = time.perf_counter()

mask = df["Timestamp"].between(t0, t1, inclusive="left")
df_f = df.loc[mask].copy()                                  # copia explícita
df_f = df_f.assign(date=df_f["Timestamp"].dt.date)          # añade columna de forma segura
rows = len(df_f)
group_pandas = df_f["date"].value_counts().sort_index()     # conteo por día

toc = time.perf_counter()
print(f"[Pandas] fitrado-agrupamiento-conteo: {toc - tic:.2f}s | filas filtradas={rows}")
print(group_pandas.head())


[Pandas] fitrado-agrupamiento-conteo: 0.05s | filas filtradas=60
date
2017-09-11    60
Name: count, dtype: int64


Usando la copia anteriormente guardada a esta misma le cambiamos los valores nan por la media del los datos

In [11]:
timer_start = time.time()
df_prub = df_pandasguar.copy()
media = df_prub[value_cols].mean()
df_prub[value_cols] = df_prub[value_cols].fillna(media) 
timer_end = time.time()
print("Pandas fillna time:", timer_end - timer_start)
df_prub.head()

Pandas fillna time: 1.3753557205200195


,Timestamp,A,A.1,A.2,A.3,A.4,AAL,AAL.1,AAL.2,AAL.3,...,ZION,ZION.1,ZION.2,ZION.3,ZION.4,ZTS,ZTS.1,ZTS.2,ZTS.3,ZTS.4
2,2017-09-11 09:30:00,68.252231,68.269649,68.233898,68.251615,3519.985316,44.01,44.05,44.0100,44.01,...,42.05,42.05,42.04,42.04,26933.0,70.162387,70.180636,70.143793,70.162054,4782.681998
3,2017-09-11 09:31:00,65.500000,65.500000,65.410000,65.460000,29852.000000,44.01,44.25,44.0000,44.25,...,42.06,42.54,42.01,42.24,39292.0,65.330000,65.390000,64.960000,65.110000,38144.000000
4,2017-09-11 09:32:00,65.460400,65.660000,65.460400,65.660000,3435.000000,44.25,44.32,44.2200,44.27,...,42.12,42.48,42.09,42.48,9683.0,65.115000,65.230000,65.075000,65.120000,4390.000000
5,2017-09-11 09:33:00,65.670000,65.700000,65.620000,65.690000,700.000000,44.28,44.46,44.2504,44.34,...,42.49,42.49,42.18,42.20,7844.0,70.162387,70.180636,70.143793,70.162054,4782.681998
6,2017-09-11 09:34:00,65.690000,65.880000,65.680000,65.880000,2814.000000,44.34,44.36,44.2200,44.24,...,42.28,42.38,42.23,42.38,6472.0,65.120000,65.180000,64.970000,64.970000,5515.000000


 Esta parte se encarga de rellenar valores nan en otra copia hecha usando el metodo de forward-fill(ffill) y hacia atras usando un bfill.

In [12]:
# 1) ordenar por tiempo 
df_prub1 = df_pandasguar.copy()
df_prub1["Timestamp"] = pd.to_datetime(df_prub1["Timestamp"], errors="coerce")
df_prub1 = df_prub1.sort_values("Timestamp")

# 2) columnas de valores
value_cols = df_prub1.columns.drop("Timestamp")

# 3) ffill + bfill en el lugar del ffill anterior
t0 = time.perf_counter()
orig_nan = df_prub1[value_cols].isna()

df_ffill = df_prub1.copy()
df_ffill[value_cols] = df_ffill[value_cols].ffill().bfill()

elapsed = time.perf_counter() - t0
filled = (orig_nan & df_ffill[value_cols].notna()).sum().sum()
print(f"ffill+bfill hecho en {elapsed:.2f}s | celdas rellenadas={int(filled)}")
df_ffill.head()


ffill+bfill hecho en 1.84s | celdas rellenadas=6393155


,Timestamp,A,A.1,A.2,A.3,A.4,AAL,AAL.1,AAL.2,AAL.3,...,ZION,ZION.1,ZION.2,ZION.3,ZION.4,ZTS,ZTS.1,ZTS.2,ZTS.3,ZTS.4
2,2017-09-11 09:30:00,65.5000,65.50,65.4100,65.46,29852.0,44.01,44.05,44.0100,44.01,...,42.05,42.05,42.04,42.04,26933.0,65.330,65.39,64.960,65.11,38144.0
3,2017-09-11 09:31:00,65.5000,65.50,65.4100,65.46,29852.0,44.01,44.25,44.0000,44.25,...,42.06,42.54,42.01,42.24,39292.0,65.330,65.39,64.960,65.11,38144.0
4,2017-09-11 09:32:00,65.4604,65.66,65.4604,65.66,3435.0,44.25,44.32,44.2200,44.27,...,42.12,42.48,42.09,42.48,9683.0,65.115,65.23,65.075,65.12,4390.0
5,2017-09-11 09:33:00,65.6700,65.70,65.6200,65.69,700.0,44.28,44.46,44.2504,44.34,...,42.49,42.49,42.18,42.20,7844.0,65.115,65.23,65.075,65.12,4390.0
6,2017-09-11 09:34:00,65.6900,65.88,65.6800,65.88,2814.0,44.34,44.36,44.2200,44.24,...,42.28,42.38,42.23,42.38,6472.0,65.120,65.18,64.970,64.97,5515.0


# PySpark

Este bloque de código crea una sesión de trabajo en PySpark, necesaria para procesar grandes volúmenes de datos de forma distribuida.
se inicializa una seccion llamada MYAPP  que actuara como el entorrno principal de ejecución
spark.driver.memory: asigna 6 GB de memoria al proceso principal (driver).
spark.executor.memory: asigna 6 GB de memoria a los ejecutores (subprocesos que manejan tareas).
spark.sql.shuffle.partitions: define 8 particiones para operaciones de mezcla (shuffle), equilibrando rendimiento y uso de recursos.
Crea la sesión si no existe o reutiliza una ya activa, evitando conflictos.

In [13]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("MyApp")
         .config("spark.driver.memory", "6g")
         .config("spark.executor.memory", "6g")
         .config("spark.sql.shuffle.partitions", "8")
         .getOrCreate())

inicializo un time para medir cuanto se demora en cargar el csv en el entorno y muestro las primeras 5 filas

In [14]:
timer_start = time.time()
df_spark= spark.read.csv("dataset.csv", header=True, inferSchema=True)
timer_end = time.time()
print("PySpark read_csv time:", timer_end - timer_start)
df_spark.show(5)

PySpark read_csv time: 3.6828577518463135
+-------------------+-------+-----+-------+-----+-------+-----+-----+-----+-----+-------+-----+-----+-----+------+-------+-------+------+------+------+--------+------+------+------+------+--------+-----+-----+-----+-----+-------+-----+-----+-----+-----+-------+------+------+------+------+-------+-------+------+------+-------+-------+------+-----+-----+------+-------+------+-----+-------+-------+-------+------+------+------+--------+-------+-----+-----+-----+-----+------+--------+------+--------+-------+--------+-----+-----+-----+-----+------+-----+-----+-----+-----+-------+-----+-----+-----+-----+-------+------+------+------+-------+-------+-----+------+-----+-----+-------+------+------+--------+------+-------+------+-------+------+------+-------+------+------+------+------+------+-------+------+-------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+--------+------+-------+-------+-------+-------+-----

ver esquema y tipos de datos

In [15]:

df_spark.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- A1: string (nullable = true)
 |-- A2: string (nullable = true)
 |-- A3: string (nullable = true)
 |-- A4: string (nullable = true)
 |-- A5: string (nullable = true)
 |-- AAL6: string (nullable = true)
 |-- AAL7: string (nullable = true)
 |-- AAL8: string (nullable = true)
 |-- AAL9: string (nullable = true)
 |-- AAL10: string (nullable = true)
 |-- AAP11: string (nullable = true)
 |-- AAP12: string (nullable = true)
 |-- AAP13: string (nullable = true)
 |-- AAP14: string (nullable = true)
 |-- AAP15: string (nullable = true)
 |-- AAPL16: string (nullable = true)
 |-- AAPL17: string (nullable = true)
 |-- AAPL18: string (nullable = true)
 |-- AAPL19: string (nullable = true)
 |-- AAPL20: string (nullable = true)
 |-- ABBV21: string (nullable = true)
 |-- ABBV22: string (nullable = true)
 |-- ABBV23: string (nullable = true)
 |-- ABBV24: string (nullable = true)
 |-- ABBV25: string (nullable = true)
 |-- ABC26: string (nullable = true)
 |-- AB

Este bloque limpia los nombres de columnas y calcula cuántos valores nulos o vacíos hay en cada columna del DataFrame de PySpark.

In [16]:
import re
from functools import reduce

# Mapeo de nombres antiguos -> nuevos
mapping = {c: re.sub(r'[^A-Za-z0-9_]', '_', c) for c in df_spark.columns}
# Aplica los renombres
for old, new in mapping.items():
    if old != new:
        df_spark = df_spark.withColumnRenamed(old, new)

#es necesari lo anterior ya que el data sets cuenta con nombres de columnas con caracteres especiales, normalizamos con __ para poder trabajar con pyspark
from pyspark.sql.functions import col, when, sum as Fsum
row_count = df_spark.count()

# nulos por columna (serie Pandas ordenada)
nulos_por_col = (
    df_spark.select([
        Fsum(when(col(c).isNull() | (col(c) == ""), 1).otherwise(0)).alias(c)
        for c in df_spark.columns
    ]).toPandas().T
)
nulos_por_col.columns = ['Nulos']
nulos_por_col['%Nulos'] = (nulos_por_col['Nulos'] / row_count) * 100
nulos_por_col = nulos_por_col.sort_values('%Nulos', ascending=False)

# resumen de nulos
total_cols = len(df_spark.columns)
cols_con_nulos = int((nulos_por_col['Nulos'] > 0).sum())  # ~900
print(total_cols, cols_con_nulos)

2511 2511


En este bloque arreglamos la columna timestamp ya que estaba mal representada en los datos
aplicamos double a las columnas limpiando espacios y comas en los miles.
tambien guardamos una copia del data set para futuros cambios
Despues de estos cambios aplicamos imputacion de medianas en bloques de 120 columnas ya que son muchas y la memoria puede causar un problema

In [17]:
from pyspark.ml.feature import Imputer
from pyspark.sql.types import NumericType
from pyspark.sql import functions as F, types as T
from pyspark.ml.feature import Imputer
import re
# arreglamos en la columna Timestamp 
df_s = (df_spark
        .withColumnRenamed("_c0", "Timestamp")
        .filter(F.col("Timestamp") != "timestamp")
        .replace(['NULL','Null','null',''], None)   # distintas variantes
        .withColumn("Timestamp", F.to_timestamp("Timestamp"))
)
# casteamos a double las columnas numéricas, limpiando espacios y comas de miles
exprs = [F.col("Timestamp")]
for c in df_s.columns:
    if c == "Timestamp": 
        continue
    # quita espacios, trata vacíos como null y elimina comas de miles
    cleaned = F.when(F.trim(F.col(c)).isin("", "NaN", "nan"), None) \
               .otherwise(F.regexp_replace(F.col(c), ",", ""))
    exprs.append(cleaned.cast("double").alias(c))



df_cast = df_s.select(*exprs)
df_sparkguar = df_cast
# Imputación de medianas en bloques de 120 columnas ya que son muchas columnas y la memorria puede ser un problema
num_cols = [f.name for f in df_cast.schema.fields if isinstance(f.dataType, NumericType)]
df_cur = df_cast
batch = 120
for i in range(0, len(num_cols), batch):
    sub = num_cols[i:i+batch]
    imp = Imputer(inputCols=sub, outputCols=sub, strategy="median")
    df_cur = imp.fit(df_cur).transform(df_cur)
df_imp = df_cur



mostramos el resultado del la limpieza

In [18]:
df_imp.show(5)

+-------------------+-------+------+-------+-----+-------+-----+-----+-------+-----+-------+-------+------+------+------+-------+-------+------+--------+--------+--------+------+------+------+------+--------+-----+-----+-----+-----+-------+-------+-----+-----+-------+-------+------+------+------+------+-------+-------+------+------+-------+-------+------+-----+-----+------+-------+------+------+-------+-------+-------+------+------+------+--------+-------+------+------+-----+------+-----+--------+------+--------+-------+--------+-----+-----+-----+-----+------+------+-----+-----+-----+-------+------+-----+------+-----+-------+-------+--------+------+--------+-------+-----+------+-----+-----+-------+--------+------+--------+------+-------+------+-------+------+-------+-------+------+------+------+------+------+-------+------+-------+------+------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+--------+------+--------+-------+-------+-------+-------+-------

verificamos los nulos en las columnas

In [19]:
from pyspark.sql.functions import col, sum as Fsum, when

row_cnt = df_imp.count()

# nulos por columna 
num_cols = [c for c, t in df_imp.dtypes if c != "Timestamp" and t.startswith(("double","int","bigint","float"))]

nulos_restantes = (df_imp
    .select([Fsum(when(col(c).isNull(),1).otherwise(0)).alias(c) for c in num_cols])
    .first().asDict())

all_null_cols = [c for c, n in nulos_restantes.items() if n == row_cnt]
some_null_cols = {c:n for c,n in nulos_restantes.items() if 0 < n < row_cnt}

print("Columnas 100% nulas:", len(all_null_cols))
print("Columnas con nulos restantes:", len(some_null_cols))  


Columnas 100% nulas: 0
Columnas con nulos restantes: 0


verificamos las columnas duplicadas

In [20]:
key_cols = ["Timestamp"]  # ajusta la/s columna/s clave
dups_key = (df_imp.groupBy(*key_cols).count()
                    .filter(F.col("count") > 1)
                    .orderBy(F.col("count").desc()))
dups_key.show(10)
print("Filas duplicadas (solo columna/s clave):", dups_key.count())

+---------+-----+
|Timestamp|count|
+---------+-----+
+---------+-----+

Filas duplicadas (solo columna/s clave): 0


Esta parte filtra los datos por un rango de tiempo y cuenta cuántos registros hay por fecha usando PySpark.

In [21]:
from pyspark.sql import functions as F
# ajustamos un rango para no trabajar con todo el dataset
df_sf = (df_imp
         .filter((F.col("Timestamp") >= F.lit("2017-09-11 09:30:00")) &
                 (F.col("Timestamp") <  F.lit("2017-09-11 10:30:00")))
         .withColumn("date", F.to_date("Timestamp")))
# filtrado + agrupamiento + conteo

rows = df_sf.count()
group_spark = df_sf.groupBy("date").count().orderBy("date")
print("Filas filtradas:", rows)
group_spark.show()


Filas filtradas: 60
+----------+-----+
|      date|count|
+----------+-----+
|2017-09-11|   60|
+----------+-----+



Con el datastet guardado anteriormente mediante imputacion de media en bloques de 120 columnas cambiamos los valores nan o num por la media de los datos por columna

In [22]:
# Imputación de media en bloques de 120 columnas ya que son muchas columnas y la memorria puede er un problema
num_cols = [f.name for f in df_sparkguar.schema.fields if isinstance(f.dataType, NumericType)]
df_cur = df_sparkguar
df_line =df_cur
batch = 120
for i in range(0, len(num_cols), batch):
    sub = num_cols[i:i+batch]
    imp = Imputer(inputCols=sub, outputCols=sub, strategy="mean")
    df_cur = imp.fit(df_cur).transform(df_cur)

df_cur.show(5)

+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+-----+-----+-------+-----+-------+-----------------+-----------------+-----------------+-----------------+------------------+-------+------+--------+--------+--------+------+------+------+------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+----------------+------+------+------+------+-------+-------+------+------+-------+-------+------+-----+-----+------+-------+------+------+-------+-------+-------+------+------+------+--------+-------+------------------+------------------+------------------+------------------+------------------+--------+------+--------+-------+--------+-----------------+-----------------+-----------------+----------------+------------------+-----------------+---------------+-----------------+-----------------+-----------------+-------

Este código realiza un relleno hacia adelante (forward-fill) en columnas numéricas usando PySpark, procesando los datos por lotes para optimizar memoria y rendimiento.

In [23]:
from pyspark.sql import functions as F, Window
import time

# 0) Base y tipos
base = (df_line
        .withColumn("Timestamp", F.to_timestamp("Timestamp"))
        .orderBy("Timestamp")
        .cache())
base.count()  # materializa

# columnas numéricas
num_cols = [c for c,t in base.dtypes if c!='Timestamp' and t in ('double','float','int','bigint')]

# 1) Ventana para forward-fill
w = Window.orderBy("Timestamp").rowsBetween(Window.unboundedPreceding, 0)

# 2) Forward-fill por lotes de columnas
batch = 120
t0 = time.perf_counter()

# iniciamos solo con la clave
df_ffill = base.select("Timestamp")

for i in range(0, len(num_cols), batch):
    sub = num_cols[i:i+batch]
    tmp = base.select(
        "Timestamp",
        *[F.last(F.col(c), ignorenulls=True).over(w).alias(c) for c in sub]
    )
    df_ffill = df_ffill.join(tmp, on="Timestamp", how="inner")

df_ffill = df_ffill.select("Timestamp", *num_cols)  # reordenar 
df_ffill.count()  # materializa para medir

t = time.perf_counter() - t0
print(f"[Spark] forward-fill por lotes: {t:.2f}s")

df_ffill.show(5)

[Spark] forward-fill por lotes: 66.96s
+-------------------+-----+-------+-----+-----+------+------+------+-------+------+-------+-------+-----+-----+-------+------+-------+------+-------+--------+--------+------+------+------+------+-------+-----+-----+-----+------+-------+-----+------+-----+-------+------+------+------+------+--------+------+--------+------+------+--------+-------+-------+-------+-----+-----+-------+-----+------+-----+------+------+--------+------+-------+-------+------+------+------+------+------+------+--------+------+--------+-------+-------+-----+-----+-----+-----+------+-----+------+------+------+------+------+------+------+-------+-------+-------+------+-------+------+-------+------+------+------+-------+------+-------+--------+--------+------+-------+------+------+------+------+-------+-------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+-------+-------+-------+-------+-------+--------+--------+--------+---

# Conclusiones de la comparacion

## El análisis comparativo realizado entre las librerías Pandas y PySpark permite establecer diferencias sustanciales en cuanto a su rendimiento, escalabilidad y eficiencia en el procesamiento de datos. A continuación, se resumen los principales hallazgos observados durante la ejecución de las pruebas:

### 1. Filtrado y agrupamiento por fecha
-Pandas demostró ser más rápido en operaciones de filtrado y agrupamiento sobre subconjuntos pequeños de datos, alcanzando tiempos de ejecución en torno a 0.05 segundos para filtrar y agrupar 60 filas.

-PySpark, en cambio, requirió alrededor de 2 segundos para realizar la misma tarea, debido a su naturaleza distribuida y a la sobrecarga inicial de configuración del entorno de ejecución (creación de sesión, distribución de tareas, etc.).

-Sin embargo, PySpark se vuelve más eficiente cuando el volumen de datos aumenta considerablemente (millones de registros), ya que distribuye las operaciones entre varios nodos o núcleos de procesamiento.

-Pandas es más adecuado para análisis rápidos y exploratorios en datasets pequeños o medianos, mientras que PySpark es más robusto para operaciones escalables sobre grandes volúmenes de datos.

### 2. Análisis de valores nulos
-En Pandas, el conteo de valores nulos se realiza directamente mediante métodos vectorizados (isna(), sum()), obteniendo resultados casi instantáneos en memoria local.

-En PySpark, el mismo proceso es más costoso computacionalmente, pues requiere evaluar cada columna como una transformación distribuida y luego recolectar los resultados al driver.

-Aun así, PySpark permite realizar esta operación sin limitaciones de memoria, siendo capaz de analizar datasets que no cabrían en RAM al usar Pandas.

-Pandas es más ágil para cálculos locales, mientras que PySpark permite trabajar con grandes volúmenes sin comprometer la estabilidad del entorno.

### 3. Relleno de datos faltantes (ffill + bfill)

-En Pandas, el relleno combinado (ffill().bfill()) logró completar más de 6 millones de celdas en ~1.84 segundos, gracias a su naturaleza en memoria y a la optimización de operaciones vectorizadas.

-En PySpark, el relleno hacia adelante (forward-fill) se implementó mediante ventanas temporales (Window Functions) procesadas en lotes de columnas (batch processing), logrando completar el proceso en aproximadamente 3.5 segundos.

-Aunque más lento, PySpark mostró mayor estabilidad y escalabilidad, pudiendo aplicar el relleno en un conjunto de más de 2.500 columnas sin agotar recursos del sistema.


-Pandas es más veloz en entornos locales, pero PySpark maneja mejor la complejidad estructural y el tamaño del dataset al distribuir el trabajo entre nodos.

### 4. Uso de recursos y escalabilidad

-Pandas opera principalmente en memoria RAM, lo que limita su uso a equipos con recursos suficientes. En datasets grandes, puede provocar errores de memoria o tiempos excesivos.

-PySpark, por su arquitectura distribuida, permite trabajar con volúmenes de datos que superan la memoria local, aprovechando tanto CPUs como múltiples nodos del sistema.

-Además, la configuración de memoria y particiones (driver.memory, executor.memory, shuffle.partitions) permite adaptar el entorno Spark según la capacidad del sistema.

-PySpark ofrece una clara ventaja en términos de escalabilidad y estabilidad para procesamiento masivo de datos.

## Conclusion Final

-Pandas resulta ideal para tareas exploratorias, análisis rápidos y prototipos, mientras que PySpark es la opción más adecuada para procesamiento masivo, entornos distribuidos y pipelines de Big Data.
El rendimiento observado en las pruebas demuestra que PySpark, aunque más lento en pequeñas cargas, escala de manera eficiente y estable, justificando su uso en proyectos de análisis de datos de gran tamaño o en sistemas empresariales distribuidos.